In [1]:
import torchvision.models as models

model = models.resnet50(weights=None)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
from torchvision.models import vgg16_bn, VGG16_BN_Weights
import torch.nn as nn
import torchsummary

def get_pretrained_model(n_classes, freezed=True):
    model = vgg16_bn(weights=VGG16_BN_Weights.DEFAULT)
    for param in model.parameters():
        param.requires_grad = not freezed

    model.classifier = nn.Sequential(
        nn.Linear(512*7*7, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, n_classes),)
    return model

model = get_pretrained_model(n_classes=37, freezed=True).cuda()
torchsummary.summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [ ]:
## Feature Extraction
model = get_pretrained_model('vgg16', n_classes=37, freezed=True).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
metrics = {'acc': accuracy}

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics)
clf.fit(train_loader, n_epochs=5, valid_loader=valid_loader)

## Fine Tuning
for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics)
clf.fit(train_loader, n_epochs=100, valid_loader=valid_loader)

## Training Hold-out Data
clf = TrainerWithMetrics(model, optimizer, loss_fn, metrics)
clf.fit(valid_loader, n_epochs=3)

In [ ]:
from torchvision.models import vgg16_bn, VGG16_BN_Weights
import torch.nn as nn

def get_pretrained_model(n_classes, freezed=True):
    backbone = vgg16_bn(weights=VGG16_BN_Weights.DEFAULT)
    for param in model.parameters():
        param.requires_grad = not freezed

    backbone.classifier = nn.Sequential(
        nn.Linear(512*7*7, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Linear(256, 4), ## [x, y, w, h]
        nn.Sigmoid(),)
    return model

import torch

def mse_loss(y_pred, y_true):
    y_pred, y_true = y_pred[:, :4], y_true[:, :4]
    loss_fn = nn.MSELoss()
    return 10*loss_fn(y_pred, y_true)

def iou(box1, box2):
    x1, y1, w1, h1 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
    x2, y2, w2, h2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

    inter_w = torch.minimum(x1 + w1/2, x2 + w2/2) - torch.maximum(x1 - w1/2, x2 - w2/2)
    inter_h = torch.minimum(y1 + h1/2, y2 + h2/2) - torch.maximum(y1 - h1/2, y2 - h2/2)
    intersection = torch.maximum(inter_w, torch.zeros_like(inter_w)) \
                    * torch.maximum(inter_h, torch.zeros_like(inter_h))
    iou = intersection / (w1*h1 + w2*h2 - intersection)
    return iou.mean()

In [ ]:
import segmentation_models_pytorch as smp

def iou_score(y_pred, y_true, n_classes=3):
    tp, fp, fn, tn = smp.metrics.get_stats(y_pred, y_true, mode='multilabel', threshold=0.5)
    return smp.metrics.iou_score(tp, fp, fn, tn, reduction='micro')

def f1_score(y_pred, y_true, n_classes=3):
    tp, fp, fn, tn = smp.metrics.get_stats(y_pred, y_true, mode='multilabel', threshold=0.5)
    return smp.metrics.f1_score(tp, fp, fn, tn, reduction='micro')

def bal_accuracy(y_pred, y_true, n_classes=3):
    tp, fp, fn, tn = smp.metrics.get_stats(y_pred, y_true, mode='multilabel', threshold=0.5)
    return smp.metrics.balanced_accuracy(tp, fp, fn, tn, reduction='micro')


model = smp.Unet('vgg16_bn', in_channels=3, classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = smp.losses.DiceLoss(mode='multilabel')
metrics={'iou': iou_score, 'f1': f1_score, 'acc': bal_accuracy}

clf = TrainerWithEarlyStopping(model, optimizer, loss_fn, metrics)
hist = clf.fit(train_loader, n_epochs=5, valid_loader=valid_loader)